# Token House Votes

In [ ]:
import pandas as pd

In [2]:
df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
df_vote_cast = pd.read_csv("./vote_cast.csv")
df_proposals = pd.read_csv("./proposals.csv")

In [ ]:
df_proposals_votes = pd.read_csv("./raw_data/2024-06-28_optimism_proposals_votes.csv")

In [15]:
df_onchain = df_proposals_votes.query('vote_type == "ON_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_onchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200 = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)

df_top_200['reason'] = df_top_200['reason'].fillna('NA')

In [21]:
df_top_200.to_csv("./proposals_votes.csv")

# Citizen House Votes

In [80]:
import requests

# df_citizen_snapshot_proposal = pd.read_json('citizen_snapshot_proposals.json')

url = "https://hub.snapshot.org/graphql?operationName=Proposals&query=query%20Proposals%20%7B%0A%20%20proposals%20(%0A%20%20%20%20first%3A%20100%2C%0A%20%20%20%20skip%3A%200%2C%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20space_in%3A%20%5B%22citizenshouse.eth%22%5D%2C%0A%20%20%20%20%20%20state%3A%20%22closed%22%0A%20%20%20%20%7D%2C%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20title%0A%20%20%20%20body%0A%20%20%20%20choices%0A%20%20%20%20start%0A%20%20%20%20end%0A%20%20%20%20snapshot%0A%20%20%20%20state%0A%20%20%20%20author%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%20%20name%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D"
response = requests.get(url)
df_citizen_snapshot_proposal=pd.DataFrame(response.json()['data']['proposals'])
df_citizen_snapshot_proposal=df_citizen_snapshot_proposal.rename(columns={'id':'proposal_id'})


In [84]:
df_citizen_snapshot_proposal_votes = pd.DataFrame()

for id in df_citizen_snapshot_proposal.proposal_id.to_list():
    url = f"https://hub.snapshot.org/graphql?operationName=Votes&query=query%20Votes%20%7B%0A%20%20votes%20(%0A%20%20%20%20first%3A%201000%0A%20%20%20%20skip%3A%200%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20proposal%3A%20%22{id}%22%0A%20%20%20%20%7D%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20voter%0A%20%20%20%20created%0A%20%20%20%20proposal%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%20%20choice%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"
    response = requests.get(url)
    temp = response.json()['data']['votes']
    for t in range(len(temp)):
        temp[t]['proposal_id'] = temp[t]['proposal']['id']
        temp[t]['space_id'] = temp[t]['space']['id']
    temp_df = pd.DataFrame(temp)
    df_citizen_snapshot_proposal_votes = pd.concat([df_citizen_snapshot_proposal_votes,temp_df])

df_citizen_snapshot_proposal_votes = pd.merge(df_citizen_snapshot_proposal_votes,df_citizen_snapshot_proposal,how="left",on='proposal_id')


In [85]:
df_citizen_snapshot_proposal_votes.to_csv('./citizen_snapshot_proposal_votes.csv')